# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import hyperspy.api as hs
from snmfem.estimators import NMF
import snmfem.datasets as ds

import matplotlib.pyplot as plt
import numpy as np

from snmfem.models.EDXS_function import print_concentrations_from_P

# Generating artificial datasets and loading them

If the datasets were already generated, they are not generated again

**This is optional**

In [ ]:
ds.generate_built_in_datasets(seeds_range=5)
spim = ds.load_particules(sample = 0)

# Loading your data and settings

There are three things to fill here : 
- An input filename in the ``hs.load()`` function
- The acquisition parameters in the ``spim.set_analysis_parameters()`` function. Be careful the previous metadata are overwritten. (I still need to build a function that does not overwrite the already present settings)
- if required you can set the concentrations of some elements in some phases with the function ``spim.set_fixed_P()``. The name of the phases can be changed.

In [ ]:
spim = hs.load("filename.hspy")
spim.set_signal_type("EDXSsnmfem")
spim.set_analysis_parameters(beam_energy = 200, azimuth_angle = 0.0, elevation_angle = 22.0, tilt_stage = 0.0, elements = ["Si","Mg","Fe"], thickness = 200e-7, density = 3.5, detector_type = "SDD_efficiency.txt", width_slope = 0.01, width_intercept = 0.065, xray_db = "default_xrays.json")
spim.set_fixed_P({"p0" : {"Si" : 0.0},"p1" : {"Fe" : 0.0}, "p2" : {"Mg" : 0.0}})

# Problem solving

Full hyperspy syntax

## Loading analysis parameters

In [ ]:
G = spim.build_G("bremsstrahlung", norm = True)
shape_2d = spim.shape_2d
# phases = spim.phases
# weights = spim.weights

est = NMF( n_components = 3,tol=0.000001, max_iter = 1000, G = G, shape_2d = shape_2d,fixed_P = P,hspy_comp = True) #  true_D = phases, true_A = weights

## Calculating the decomposition

/!\ Depending on the parameters you choose and the size of the data it might take a while

In [ ]:
out = spim.decomposition(algorithm = est, return_info=True)

## Getting the losses and the results of the decomposition

- First cell : Printing the resulting concentrations.
- Second cell : Ploting the resulting spectra
- Thrid cell : Ploting the resulting abundances

In [ ]:
print_concentrations_from_P(est.P_, elements = spim.metadata.Sample.elements)

In [ ]:
spim.plot_decomposition_loadings(3)

In [ ]:
spim.plot_decomposition_factors(3)

# The remaining part is for tests for Nathanaël and I

# Problem Solving

With the usual scikit use

In [ ]:
G = spim.build_G("bremsstrahlung")
shape_2d = spim.shape_2d
phases, weights = spim.phases, spim.weights
X = spim.X

est = SmoothNMF(n_components = 3,tol=0.1, G = G, shape_2d = shape_2d, lambda_L= 2, true_D = phases, true_A = weights, hspy_comp = False)



In [ ]:
D = est.fit_transform(X)